In [ ]:
import timeit
import random
import pickle #save results
from itertools import chain
import geopandas as gpd
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
from shapely import MultiLineString
from shapely import Point

##  1, Read Data

### 1.1, Raod Network

In [ ]:
road = gpd.read_file('data/Road_Cleaned.shp')
road.head()

### 1.2, Demographic data (Census Tract)

In [ ]:
dp = gpd.read_file('data/Erie_County_Census_Tract.shp')#.set_index('GEOID10')
#check size of census track data
len(dp)

In [ ]:
dp.head()

### 1.3, Commute Information

In [ ]:
#od = pd.read_csv('Data/tract-od.csv.zip',dtype={i:(str if i<2 else int) for i in range(6)})
#od = pd.read_csv('Data/tract-od_2019.csv')
#od = pd.read_csv('Data/tract-od_2019_temp.csv',dtype={i:(str if i<2 else int) for i in range(6)})
od = pd.read_csv('data/erie-tract-od.csv')
#od = pd.read_csv('Data/ny-tract-od.csv')
od = od.astype({"work": str, "home": str})
od.head()

In [ ]:
od.dtypes

In [ ]:
od[(od.work == '36029000110') & od.home.str.startswith('36029')]

In [ ]:
len(od)

### 1.4, Workplace

In [ ]:
cbp = pd.read_csv('data/CBP2020/CBP2020.CB2000CBP-Data.csv')#.iloc[1:,[0,9,11]]
cbp = cbp[cbp['EMPSZES_LABEL'] == 'All establishments'].copy()
cbp['county'] = cbp.GEO_ID.astype(str).str[-5:]


In [ ]:
cbp.head()

In [ ]:
county_com_total = cbp[cbp['NAICS2017'] == '00'].reset_index(drop=True).copy()
len(county_com_total)

In [ ]:
county_com_total.head()

## 2, Data PreProcessing
#### Add 3 columns on dp (demographics) dataframe
* [col1] controls the number of individuals will be generated within each census tract
* [col2] the number of workplaces
* [col3] the probability for employees(individuals) to be assigned in each workplace

### 2.2 The number of workplaces

In [ ]:
def number_of_wp(data, od, cbp):
    """
    calculate number of workplaces for each tract
    wp_tract = wp_cty * (tract_employed / county_employed)
    """
    try:
        """
        calculate number of workplaces for each tract
        wp_tract = wp_cty * (tract_employed / county_employed)
        """

        #print("++++++++++")
        # use "od data" to get number of jobs in each census tract
        # and number of jobs in each county
        tract_number = data.GEOID10#.astype(int)
        tract_jobs_df = od[['work','S000']].groupby('work').sum().reset_index()
        tract_jobs_df = tract_jobs_df.astype({"work": str})
        tract_jobs_number = int(tract_jobs_df.loc[tract_jobs_df.work == tract_number, 'S000'].values[0]) # total number of job in each tract
        #print("Tract", tract_number, "has", tract_jobs_number, "jobs")

        #county total number of job
        county_number = tract_number[:5]
        tract_jobs_df['county'] = tract_jobs_df.work.astype(str).str[:5]
        county_job_df = tract_jobs_df[['county','S000']].groupby('county').sum().reset_index()
        county_job_number = int(county_job_df.S000[0])
        #print("County", county_number, "has", county_job_number, "jobs")
        #print("++++++++++")

        #use company number
        #county wrk place establishment number
        #cbp['county'] = cbp.GEO_ID.astype(str).str[-5:]
        #print("==========")
        campany_df = cbp.loc[:,['county', 'NAME', 'ESTAB']]
        county_campany_number = int(campany_df[campany_df.county == county_number].ESTAB.values[0])
        #print("County", county_number, "has", county_campany_number, "company")

        tract_wp_number = int(county_campany_number * (tract_jobs_number / county_job_number))

        #print("Tract",  tract_number, "has", tract_wp_number, "companys")
        return tract_wp_number
    except:
        print(tract_number, "has issue")
        return 0

### 2.3, The probability for employees(individuals) to be assigned in each workplace

In [ ]:
def wp_proba(x):
    """
    probability of an employee working in that workplace is lognormal:
    http://www.haas.berkeley.edu/faculty/pdf/wallace_dynamics.pdf
    """
    if x == 0: return np.zeros(0)
    b = np.random.lognormal(mean=2,size=x).reshape(-1, 1)
    return np.sort(normalize(b,norm='l1',axis=0).ravel())

### 2.4 Apply Preprocessing

In [ ]:
#import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning)
#This will ignore all DeprecationWarning warnings in your code.
dp['WP_CNT'] = dp.apply(number_of_wp, args=(od, county_com_total),axis=1)
dp.head()

In [ ]:
dp.head()

In [ ]:
dp.to_file('data/Erie_Tracts_with_Work.shp', driver = "ESRI Shapefile")
dp.head()

In [ ]:

#dp = gpd.read_file('Data/Erie_Tracts_with_Work.shp')
dp = gpd.read_file('data/fairfax2010-complete.shp')
dp.head()

In [ ]:
dp.iloc[:,23:59]

In [ ]:
dp.iloc[:,154:169]

In [ ]:
# each track, the probability distribution for an employee to work there is lognormal
# this column has the probability for each workplace and they add up to 1. 
dp['WP_PROBA'] = dp.WP_CNT.map(wp_proba)

In [ ]:
dp.sort_index(axis=1,inplace=True)
#dp.sort_values(by = 'GEOID10')
dp = dp.set_index('GEOID10')
dp.head()

## 3, Synthesizing Population 

### 3,1 Create individuals

In [ ]:
def create_individuals(tract):
    #print("++++++++++Create People+++++++++++")
    """Generate a population of ages and sexes as a DataFrame

    Given the number of individuals for 18 age groups of each sex,
    return a two column DataFrame of age ([0,89]) and sex ('m','f')
    """
    #portion = tract.geometry.area / tract.Shape_Area # what portion of the tract is included
    #age_sex_groups = (tract[22:59].drop('DP0010039') * portion).astype(int)
    
    age_sex_groups = (tract[22:59].drop('DP0010039')).astype(int)
    
    #dfs=pd.DataFrame()
    dfs = []
    
    #code is the index(generated by the enumerate build-in function) of the list 
    #,which indicates the age and sex group
    #code 0~17 male
    #code 18~35 female
    for code,count in enumerate(age_sex_groups):
        base_age = (code % 18)*5
        gender = 'm' if code < 18 else 'f'
        ages = []
        for offset in range(4):
            ages.extend([offset+base_age]*(count//5))
        ages.extend([base_age+4]*(count-len(ages)))
        temp = pd.DataFrame({'code':code, 'age':ages,'sex':[gender]*count})
        dfs.append(pd.DataFrame({'code':code, 'age':ages,'sex':[gender]*count}))
        #dfs = pd.concat([dfs, temp])
    df = pd.concat(dfs).sample(frac=1,random_state=123).reset_index(drop=True)
    #df = pd.concat([dfs]).sample(frac=1,random_state=123).reset_index(drop=True)
    df.index = tract.name + 'i' + df.index.to_series().astype(str)
    #df['friends'] = [set()] * len(df)
    return df

### 3.2 Create households

In [ ]:
def create_households(tract, people):
    #print("++++++++++Create Hhold+++++++++++")
    #get the amount of each household type (GOOD)
    hh_cnt = get_hh_cnts(tract)
    #print(hh_cnt)
    
    #create a empty df to hold indivadual hhold 
    hholds = pd.DataFrame()
    
    #create a column to in hhold to hold the households types info
    hholds['htype'] = np.repeat(hh_cnt.index,hh_cnt)
    #print(hholds[hholds.htype == 6])
    #hholds = hholds[hholds.htype != 6].sort_values('htype',ascending=False).concat(hholds[hholds.htype == 6])
    #hholds = hholds[hholds.htype != 6].sort_values('htype',ascending=False).append(hholds[hholds.htype == 6])
    
    temp1 = hholds[hholds.htype != 6].sort_values('htype',ascending=False)
    #print(temp1)
    temp2 = hholds[hholds.htype == 6]
    #print(temp2)
    hholds = pd.concat([temp1, temp2])#.sort_values('htype',ascending=False)
    #print(hholds)
    #create member for each households; 
    #for remaining populating, populate them in households as relatives and those living in group quarter (non-household)
    populate_households(tract, people, hholds)

In [ ]:
#Get the number of household under different household types
def get_hh_cnts(tract):
    """
    Eleven household types:
    0         h&w (no<18)
    1      h&w&ch (ch<18)
    2        male (no<18)
    3        male (ch<18)
    4      female (no<18)
    5      female (ch<18)
    6     nonfamily group
    7       lone male <65
    8       lone male >65
    9      lone female<65
    10     lone female>65
    """

    householdConstraints = (tract[150:166]).astype(int) #HOUSEHOLDS BY TYPE
    #print(householdConstraints.head())
    hh_cnt = pd.Series(np.zeros(11),dtype=int) #11 household types (group quarters are not household)

    # husband/wife families
    # husband-wife family DP0130004 - DP0130005: Husband-wife family - With own children under 18 years
    hh_cnt[0] = householdConstraints[4] - householdConstraints[5]; 
    # husband-wife family DP0130005, OWN CHILDREN < 18
    hh_cnt[1] = householdConstraints[5]; 
    
    # male householders
    # single male householder DP0130006 - DP0130007: Male householder, no wife present - Male householder, no wife present
    hh_cnt[2] = householdConstraints[6] - householdConstraints[7]; 
    # single male householder DP0130007, OWN CHILDREN < 18
    hh_cnt[3] = householdConstraints[7];
    
    # female householders
    # single female householder DP0130008 - DP0130009: Female householder, no husband present - With own children under 18 years
    
    # single female householder DP0130009, OWN CHILDREN < 18
    hh_cnt[4] = householdConstraints[8] - householdConstraints[9]; # single female householder
    hh_cnt[5] = householdConstraints[9]; # single female householder, OWN CHILDREN < 18
    
    
    # nonfamily householder
    hh_cnt[6] = householdConstraints[10] - householdConstraints[11]; # nonfamily group living
    hh_cnt[7] = householdConstraints[12] - householdConstraints[13]; # lone male < 65
    hh_cnt[8] = householdConstraints[13]; # lone male >= 65
    hh_cnt[9] = householdConstraints[14] - householdConstraints[15]; # lone female < 65
    hh_cnt[10] = householdConstraints[15]; # lone female >= 65

    return hh_cnt

In [ ]:
#Generate household members based on hhold type
def gen_households(hh_type, people, mask):
    """
    Eleven household types:
    0         h&w (no<18) husband and wife no kids
    1      h&w&ch (ch<18) husband and wife with kids
    2        male (no<18) male with no kids (wife not present)
    3        male (ch<18) male with kids (wife not present)
    4      female (no<18) female with no kids (husband not present)
    5      female (ch<18) female with kids (husband not present) 
    6     nonfamily group 
    7       lone male <65 male younger than 65 lives alone 
    8       lone male >65 male older than 65 lives alone
    9      lone female<65 female younger than 65 lives alone
    10     lone female>65 female older than 65 lives alone
    """
    #create a list to hold household member
    members = []
    

    #first, create household head, 11 types
    #age range of the householder for each household type  (the range comes from dp age range above)
    head_ranges = [
                    range(4,18), range(4,14), range(4,18), range(4,14),range(22,36), range(22,30),
                    #6
                    chain(range(4,18),range(21,36)),  
                    range(4,13), range(13,18), range(21,31), range(31,36)
                  ]
 
    '''
        meaning of the head_ranges: 
        [(15,99)/m/hh0,(20,70)/m/hh1,(15,99)/m/hh2,(20,70)/m/hh3,(15,99)/f/hh0,(20,70)/f/hh1,
        (15,99)/f/hh4,(15,65)/f/hh5,(20,65)/m/hh7,(65,99)/m/hh8,(15,65)/f/hh9,(65,99)/f/hh10]
        
        head_sex:
        [(1'm'),(2'm'),(3'm'),(4'm'),(5'f'),(6'f'),(7'f'),(8'f'),(9'm'),(10'm'),(11'f'),(12'f')]

    ''' 
    #add the householder
    pot = people[mask].code #potential's age or age group
    
    #selcet households head
    iindex = pot[pot.isin(head_ranges[hh_type])].index[0] #potential's age is in the range of this hh type
    # what is hh_type
    
    h1 = people.loc[iindex] #age & sex of h1, what is h1
    
    mask[iindex] = False
    members.append(iindex)

    #if living alone then return the members
    if hh_type > 6:
        return members

    #if husband and wife, add the wife
    if hh_type in (0,1):
        pot = people[mask].code
        if h1.code == 4: #if husband is 20~24
            iindex = pot[pot.isin(range(h1.code+17,h1.code+20))].index[0] #wife is -4~14 + husband age
        else: # if husband is older than 20~24
            iindex = pot[pot.isin(range(h1.code+16,h1.code+20))].index[0] 
        
        h2 = people.loc[iindex] # -4 < husband.age - wife.age < 9
        mask[iindex] = False
        members.append(iindex)


    """A child includes a son or daughter by birth (biological child), a stepchild,
    or an adopted child of the householder, regardless of the child’s age or marital status.
    The category excludes sons-in-law, daughters- in-law, and foster children."""
    #household types with at least one child (18-)
    if hh_type in (1,3,5):
        #https://www.census.gov/hhes/families/files/graphics/FM-3.pdf
        if hh_type == 1:
            num_of_child = max(1,abs(int(np.random.normal(2)))) #gaussian touch
        elif hh_type == 3:
            num_of_child = max(1,abs(int(np.random.normal(1.6)))) #gaussian touch
        elif hh_type == 5:
            num_of_child = max(1,abs(int(np.random.normal(1.8)))) #gaussian touch

        pot = people[mask]
        if hh_type == 1:
            iindices = pot[(pot.age<18) & (45 > h2.age-pot.age)].index[:num_of_child]
        else: #father (mother) and child age difference not to exceed 50 (40)
            age_diff = 45 if hh_type == 5 else 55
            iindices = pot[(pot.age<18) & (age_diff>h1.age-pot.age)].index[:num_of_child]
        
        for i in iindices:
            child = people.loc[i]
            mask[i] = False
            members.append(i)

    #if nonfamily group then either friends or unmarried couples
    if hh_type == 6:
        pot = people[mask].code
        num_of_friends = max(1,abs(int(np.random.normal(1.3)))) #gaussian touch
        iindices = pot[pot.isin(range(h1.code-2,h1.code+3))].index[:num_of_friends]
        for i in iindices:
            friend = people.loc[i]
            mask[i] = False
            members.append(i)

    return members


def populate_households(tract, people, hholds):
    
    #What is the mask for?
    mask = pd.Series(True, index = people.index) #[True]*len(people)
    
    hholds['members'] = hholds.htype.apply(gen_households,args=(people, mask,))
    
    """The seven types of group quarters are categorized as institutional group quarters
    (correctional facilities for adults, juvenile facilities, nursing facilities/skilled-nursing facilities,
    and other institutional facilities) or noninstitutional group quarters (college/university student housing,
    military quarters, and other noninstitutional facilities)."""
    
    group_population = int(tract.DP0120014) #people living in group quarters (not in households)
    
    #gq_indices = people[(people.age>=65) | (people.age<18)].index[:group_population]
    gq_indices = people[mask].index[:group_population]
    
    #for i in gq_indices: mask[i] = False
    mask.loc[gq_indices] = False

    #now distribute the remaining household guys as relatives...
    relatives = people[mask].index
    it = iter(relatives) #sample by replacement
    relative_hhs = hholds[hholds.htype<7].sample(n=len(relatives),replace=True)
    relative_hhs.members.apply(lambda x: x.append(next(it))) #appends on mutable lists
    #for i in relatives: mask[i] = False
    mask.loc[relatives]= False
    #print('is anyone left homeless:',any(mask))
    #add those living in group quarters as all living in a house of 12th type
    if group_population > 0:
        hholds.loc[len(hholds)] = {'htype':11, 'members':gq_indices}
    
    # name households
    hholds = hholds.set_index(tract.name+'h'+pd.Series(np.arange(len(hholds)).astype(str)))

    ## where is hh???
    def hh_2_people(hh,people):
        for m in hh.members:
            people.loc[m,'hhold'] = hh.name
            people.loc[m,'htype'] = hh.htype
    
    hholds.apply(hh_2_people,args=(people,),axis=1)
    people['htype'] = people.htype.astype(int)

### 3.3 Assign workplaces 

In [ ]:
def assign_workplaces(tract, people, od):
    #print("++++++++++Assigning Work+++++++++++")
    """
    if the destination tract of a worker is not in our DP dataset
    then we assign his wp to 'DTIDw', otherwise 'DTIDw#'
    
    the actual size distribution of establishments is lognormal
    https://www.princeton.edu/~erossi/fsdae.pdf
    """
    #destination tracts and numbers
    #print("tract number is",tract.name, type(tract.name))
    td = od[od['home'] == tract.name].set_index('work').S000
    
    td = td.apply(np.ceil).astype(int) #from this tract to others
    # 58.5%: US population (16+) - employment rate
    # https://data.bls.gov/timeseries/LNS12300000
    employed = people[people.age>=18].sample(td.sum()).index #get the employed
    dtract = pd.Series(np.repeat(td.index.values, td.values)) #get the destination tract
    # if 'wp' in people.columns: people.drop('wp',axis=1,inplace=True)
    people.loc[employed,'wp'] = dtract.apply(lambda x: x+'w'+str(np.random.choice(dp.loc[x,'WP_CNT'],p=dp.loc[x,'WP_PROBA'])) if x in dp.index else x+'w').values

### 3.4, Get errors

In [ ]:
def get_errors(tract,people):
    """Percentage errors"""
    err = {}    
    #portion = tract.geometry.area / tract.Shape_Area # what portion of the tract is included
    #senior_actual = int(tract.DP0150001 * portion) # Households with individuals 65 years and over
    senior_actual = int(tract.DP0150001)
    #minor_actual = int(tract.DP0140001 * portion) # Households with individuals under 18 years
    minor_actual = int(tract.DP0140001)
    #err['tract'] = tract.name
    err['population'] = tract.DP0010001
    err['in_gq'] = tract.DP0120014
    avg_synthetic_family = people[people.htype<6].groupby('hhold').size().mean()
    err['avg_family'] = 100*(avg_synthetic_family - tract.DP0170001) / tract.DP0170001
    err['avg_hh'] = 100*(people[people.htype!=11].groupby('hhold').size().mean() - tract.DP0160001) / tract.DP0160001 
    err['senior_hh'] = 100*(people[people.age>=65].hhold.nunique() - senior_actual) / senior_actual
    err['minor_hh'] = 100*(people[people.age<18].hhold.nunique() - minor_actual) / minor_actual
    return pd.Series(err,name=tract.name)

### 3.5, Create geo locations for each individuals

In [ ]:
# create space
#shapely geometries are not hashable, here is my hash function to check the duplicates
def hash_geom(g):
    #print("Get the lat and long of the road segment and output to tuple")
    if g.geom_type == 'MultiLineString':
        #print("MultiLineString")
        #print(g.geoms)
        cord_list = []
        for line in g.geoms:
            for lat,lon in line.coords[:]:
                #print((round(lat,6),round(lon,6)))
                cord_list.append((round(lat,6),round(lon,6)))
        #print(cord_list)
        cord_tuple = tuple(item for item in cord_list)
        #print("Tuple")
        #print(cord_tuple)
        return tuple(item for item in cord_list)
    else:
        #print("Line")
        #print(tuple((round(lat,6),round(lon,6)) for lat,lon in g.coords[:]))
        return tuple((round(lat,6),round(lon,6)) for lat,lon in g.coords[:]) #shaply older version
        #return tuple((round(lat,6),round(lon,6)) for lat,lon in g.geoms) #shapely 2.0 or later version

In [ ]:
def create_home_location(tract, hcnt, road):
    #print("Creating House Space...")
    #create houses
    if tract.DP0120014 > 0: hcnt += 1 #people living in group quarters (not in households)
    
    mask = road.intersects(tract.geometry) #get the road with in the census tract
     #home road
    hmask = mask & road.MTFCC.str.contains('S1400|S1740') #get the home road
    hrd = road[hmask].intersection(tract.geometry)#get the geometry of home road
    hrd = hrd[hrd.geom_type.isin(['LinearRing', 'LineString', 'MultiLineString'])]
    hrd = hrd[~hrd.apply(hash_geom).duplicated()] #remove the duplicate lat and long by returning boolean Series denoting duplicate rows (using .duplicated()).

    HD=0.0005
    houses = hrd.apply(lambda x: pd.Series([x.interpolate(seg) for seg in np.arange(HD,x.length,HD)]))
    #print(houses)
    houses = houses.unstack().dropna().reset_index(drop=True) #flatten
    houses = houses.sample(n=hcnt,replace=True).reset_index(drop=True)
    houses.index = tract.name + 'h' + houses.index.to_series().astype(str)
    
    return gpd.GeoSeries(houses)

In [ ]:
#road[road.MTFCC == 'S1100'].head()

In [ ]:
def create_work_location(tract, road):
    
    #from shapely import MultiLineString, Point, ops, LineString
    
    WD=0.0002
    
    mask = road.intersects(tract.geometry) #get the road with in the census tract
    """
        * Home Road
        S1400 Local Neighborhood Road, Rural Road, City Street
        S1740 Private Road for service vehicles (logging, oil fields, ranches, etc.) 
        * Work Road
        S1100 Primary Road
        S1200 Secondary Road
    """
    #work road
    wmask = mask & road.MTFCC.str.contains('S1100|S1200') #S1100 Primary Road; S1200 Secondary Road,
    wrd = road[wmask].intersection(tract.geometry)#get the geometry of work road
    wrd = wrd[wrd.geom_type.isin(['LinearRing', 'LineString', 'MultiLineString'])]
    wrd = wrd[~wrd.apply(hash_geom).duplicated()] #remove the duplicate lat and long by returning boolean Series denoting duplicate rows (using .duplicated()).
    
    #home road
    hmask = mask & road.MTFCC.str.contains('S1400|S1740') #get the home road
    hrd = road[hmask].intersection(tract.geometry)#get the geometry of home road
    hrd = hrd[hrd.geom_type.isin(['LinearRing', 'LineString', 'MultiLineString'])]
    hrd = hrd[~hrd.apply(hash_geom).duplicated()] #remove the duplicate lat and long by returning boolean Series denoting duplicate rows (using .duplicated()).
    
    wrk_point = wrd.apply(lambda x: pd.Series([x.interpolate(seg) for seg in np.arange(WD, x.length,WD)]))#Get the workplace loaction point on each road segment
    #print(wrk_point)
    #workplaces on the intersection of home roads with types S1400|S1740
    #rwps = hrd.apply(lambda x: Point(x.coords[0]) if type(x) != MultiLineString else Point(x[0].coords[0]))
    rwps = hrd.apply(lambda x: Point(x.coords[0]) if type(x) != MultiLineString else Point(x.geoms[0].coords[0]))
    #print(rwps)
    
    wrk_place = pd.DataFrame()
    #print(wrk_point)
    if len(wrk_point) > 0:
        temp = wrk_point.unstack().dropna().reset_index(drop=True)
        wrk_place = pd.concat([rwps, temp])
        #wrk_place = rwps.append(wrk_point.unstack().dropna().reset_index(drop=True))
    else:
        wrk_place = rwps
    wrk_place = wrk_place.sample(n = tract.WP_CNT,replace=True).reset_index(drop=True)
    wrk_place.index = tract.name + 'w' + wrk_place.index.to_series().astype(str)
    #print("workplace")
    #print(wrk_place)'

    return gpd.GeoSeries(wrk_place)

### 3.7 Main synthesize function

In [ ]:
def synthesize(tract, od, road, errors, population, wps):
    start_time = timeit.default_timer()
    #print(tract.name,'started...',end=' ')
    try:
        people = create_individuals(tract)
        #print(people)
        create_households(tract,people)
        #print(people)
        assign_workplaces(tract,people,od)
        #create home loaction
        houses = create_home_location(tract, people.hhold.nunique(), road)
        #create work location
        workplaces = create_work_location(tract, road)
        #houses, wp = create_spaces(tract, people.hhold.nunique(), od, road)
        people['geometry'] = people.hhold.map(houses)
        err = get_errors(tract,people)
    
        #population = pd.concat([population, people])
        population.append(people)
        #print(population)
        wps.append(workplaces)
        #wps = pd.concat([wps, workplaces])
        #print(wps)
        errors.append(err)
        #errors = pd.concat([errors, err])
    except:
        print(tract.name," has problems")
        fd = open('Results/problematic_tracts.csv','a')
        fd.write(tract.name)
        fd.close()
    #print(tract.name,'now ended ({:.1f} secs)'.format(timeit.default_timer() - start_time))

## 4, Apply above functions to generate Population

### 4.1, Test Function

In [ ]:
#list to hold population
population = []
#hold error
errors = []
#hold workplaces info
wps = []

print("Start Wokring...")

#set timer
start_time = timeit.default_timer()

#Test Code
data = dp[:5]
#data = dp[:1000]

data.apply(lambda t: synthesize(t,od,road, errors, population, wps),axis=1)

### 4.2, Main Apply-generate Population

In [ ]:
len(dp)

In [ ]:
if __name__ == '__main__':
    #list to hold population
    population = []
    #hold error
    errors = []
    #hold workplaces info
    wps = []

    print("Start Wokring...")

    #set timer
    start_time = timeit.default_timer()

    #Test Code
    data = dp

    data.apply(lambda t: synthesize(t,od,road, errors, population, wps),axis=1)

    #Hold the resutls in pickles for later ectraction, which will save memory
    with open('results/'+"errors"+"erie"+".pkl", 'wb') as f:
        pickle.dump(errors, f)
    with open('results/'+"population"+"erie"+".pkl", 'wb') as f:
        pickle.dump(population, f)
    with open('results/'+"wps"+"erie"+".pkl", 'wb') as f:
        pickle.dump(wps, f)

    elapsed = timeit.default_timer() - start_time
    print("Total Time(s):", elapsed)

### 4.1 Data extractions

In [ ]:
def CollectResultsPop(results, X):
    if X == "Pop":
        print("Population")
        temp_df = pd.DataFrame()
        for i in results:
            temp_df = pd.concat([temp_df,i])

        temp_df = temp_df.reset_index().drop(['code'], axis=1)
        temp_df.rename(columns={temp_df.columns[0]: 'id'},inplace=True)
        print(temp_df.head())

        final_df = gpd.GeoDataFrame(temp_df, geometry='geometry')
        final_df['long'] = final_df['geometry'].x
        final_df['lat'] = final_df['geometry'].y
        print("=====GP Data=====")
        print(final_df.head())
        return final_df #return pandas df with geometry
    
    if X == "Work":
        print("Work")
        wp  = pd.DataFrame()
        for j in results:
            wp = pd.concat([wp,j])
            #wp = wp.append(j, ignore_index=True)
        wp = wp.reset_index()
        wp.rename(columns={wp.columns[0]: 'wp', wp.columns[1]: 'geometry'},inplace=True)
        
        print("=====Work Data=====")
        print(wp.head())
        return wp
    
    else:
        #Errors
        print("Error")
        er = pd.DataFrame()
        for e in results:
            #print(e)
            er = pd.concat([er,e])
        #print(er.head())
        return er

In [ ]:
pop = CollectResultsPop(population, "Pop")
work = CollectResultsPop(wps, "Work")
err = CollectResultsPop(errors, "Er")

In [ ]:
pop.to_csv('results/erie_population.csv')
work.to_csv('results/erie_workplaces.csv')
#err = CollectResultsPop(errors, "Er")

In [ ]:
len(pop)

In [ ]:
pop.wp.unique()

In [ ]:
df.wp.unique()

In [ ]:
df[df.age>= 18]

In [ ]:
#Workplaces
wp  = pd.DataFrame()
for j in wps:
    wp = pd.concat([wp,j])
    #wp = wp.append(j, ignore_index=True)
wp = wp.reset_index()
wp.rename(columns={wp.columns[0]: 'wp', wp.columns[1]: 'geometry'},inplace=True)
wp.head()

In [ ]:
wp.to_csv('workplace_NY.csv')

In [ ]:
#Errors
er = pd.DataFrame()
for e in errors:
    print(e)
    er = pd.concat([er,e])
er.head()

In [ ]:
### 4.2, Save files
df.to_csv('full_population.csv')
wp.to_csv('full_workplaces.csv')
er.to_csv('full_errors.csv')